In [8]:
# %pip install langchain_chroma
# %pip install langchain_openai
# %pip install langchain_google_genai
# %pip install langchain_huggingface
# %pip install langchain_community
# %pip install FlagEmbedding
# !pip install python-dotenv
# %pip install pypdf

In [5]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.docstore.document import Document

## Load the files and index them in a vector database


In [6]:
docs= []
for doc in PyPDFDirectoryLoader("/content/data").load():


    metadata ={
        "source": doc.metadata["source"],
        "page": doc.metadata["page"],

    }
    doc_text = ' '.join(doc.page_content.split())
    # Remove newlines and extra spaces
    docs.append(Document(page_content=doc_text,
                                   metadata=metadata))

# PyPDFDirectoryLoader loads all PDFs in a directory

In [7]:
from pprint import pprint
pprint(docs[1])

Document(metadata={'source': '/content/data/gpt4.pdf', 'page': 1}, page_content='from experience. Care should be taken when using the outputs of GPT-4, particularly in contexts where reliability is important. GPT-4’s capabilities and limitations create significant and novel safety challenges, and we believe careful study of these challenges is an important area of research given the potential societal impact. This report includes an extensive system card (after the Appendix) describing some of the risks we foresee around bias, disinformation, over-reliance, privacy, cybersecurity, proliferation, and more. It also describes interventions we made to mitigate potential harms from the deployment of GPT-4, including adversarial testing with domain experts, and a model-assisted safety pipeline. 2 Scope and Limitations of this Technical Report This report focuses on the capabilities, limitations, and safety properties of GPT-4. GPT-4 is a Transformer-style model [39] pre-trained to predict th

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from dotenv import load_dotenv
import os
from langchain_chroma import Chroma
from FlagEmbedding import BGEM3FlagModel
# The splitting and chunking strategy
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv(".env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv('HUGGINGFACE_TOKEN')

In [11]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [12]:
# Download the embeddings
open_ai_large = OpenAIEmbeddings(model="text-embedding-3-large")
open_ai_small = OpenAIEmbeddings(model="text-embedding-3-small")


model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
embedding_list = [open_ai_large, open_ai_small, hf]
embedding_names = ["open_ai_large", "open_ai_small", "hf"]

In [14]:
chunked_docs = splitter.split_documents(docs)

In [ ]:

# chroma_db = Chroma.from_documents(documents=chunked_docs,
#                                   collection_name=f'capstone_db_bge',
#                                   embedding=bge,
#                                   collection_metadata={"hnsw:space": "euclidean"},
#                                 #, "bge"  persist_directory=f"./capstone_db_bge")

In [15]:
for embedding, name in zip(embedding_list, embedding_names):
    print(f"Using embedding: {name}")
    # Create a Chroma vector store
    chroma_db = Chroma.from_documents(documents=chunked_docs,
                                      collection_name=f'capstone_db_{name}',
                                      embedding=embedding,
                                      collection_metadata={"hnsw:space": "cosine"},
                                      persist_directory=f"./capstone_db")
    print(f"Chroma DB created for embedding: {name}")


Using embedding: open_ai_large
Chroma DB created for embedding: open_ai_large
Using embedding: open_ai_small
Chroma DB created for embedding: open_ai_small
Using embedding: hf
Chroma DB created for embedding: hf


In [ ]:

# chroma_db = Chroma.from_documents(documents=chunked_docs,
#                                   collection_name='capstone_db',
#                                   embedding=openaiembedding,
#                                   # need to set the distance function to cosine else it uses euclidean by default
#                                   # check https://docs.trychroma.com/guides#changing-the-distance-function
#                                   collection_metadata={"hnsw:space": "cosine"},
#                                   persist_directory="./capstone_db")

In [ ]:
# load from disk
model_name =[]
retriever_details = []
combined_list = {}

for embedding, name in zip(embedding_list, embedding_names):


    chroma_db = Chroma(persist_directory=f"./capstone_db",

                      collection_name=f'capstone_db_{name}',
                       embedding_function=embedding)
    similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 5, "score_threshold": 0.0})
    model_name.append(name)
    print(f"Model name: {model_name}")
    #print(embedding.model)
    similarity = similarity_retriever.invoke("What is attention?", k=5)
    retriever_details.append(similarity)
    print(f"Retriever details: {retriever_details}")
    details = dict(zip(model_name, retriever_details))
    combined_list.update(details)

Model name: ['open_ai_large']
Retriever details: [[Document(id='e942dafe-4e1a-4565-bf76-d54146766e57', metadata={'page': 2, 'source': '/content/data/attention_paper.pdf'}, page_content='i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3'), Document(id='17c96125-bf85-4013-b3f5-517baf8ee654', metadata={'page': 2, 'source': '/content/data/attention_paper.pdf'}, page_content='i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3'), Document(id='b267b328-a103-4fe6-a4d5-4b37a4bcc165', metadata={'page': 1, 'source': '/content/data/mistral_paper.pdf'}, page_content='we have a theoretical attention span of approximately 131K tokens. In practice, for a sequence lengt

In [ ]:
combined_list['open_ai_large'][0].page_content

'i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3'

In [ ]:
for model, retriever in combined_list.items():
    print(f"Model: {model}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}")
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Model: open_ai_large
Document: i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3
Source: /content/data/attention_paper.pdf
Page: 2
--------------------------------------------------------------------------------
Document: i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3
Source: /content/data/attention_paper.pdf
Page: 2
--------------------------------------------------------------------------------
Document: we have a theoretical attention span of approximately 131K tokens. In practice, for a sequence length of 16K andW = 4096, changes made to FlashAttention [ 11] and xFormers [ 18] yield a 2x speed improvement over a vanilla attention baseline. Rolli

### From the above embeddings, we can infer that text-embedding-3-large retrieves relevant data in the top.

### change to euclidean embeddings

In [16]:
for embedding, name in zip(embedding_list, embedding_names):
    print(f"Using embedding: {name}")
    # Create a Chroma vector store
    chroma_db = Chroma.from_documents(documents=chunked_docs,
                                      collection_name=f'capstone_db_{name}_euc',
                                      embedding=embedding,

                                      collection_metadata={"hnsw:space": "l2"},
                                      persist_directory=f"./capstone_db_euc")
    print(f"Chroma DB created for embedding: {name}")

Using embedding: open_ai_large
Chroma DB created for embedding: open_ai_large
Using embedding: open_ai_small
Chroma DB created for embedding: open_ai_small
Using embedding: hf
Chroma DB created for embedding: hf


In [ ]:
# load from disk
model_name_euc =[]
retriever_details_euc = []
combined_list_euc = {}

for embedding, name in zip(embedding_list, embedding_names):


    chroma_db = Chroma(persist_directory=f"./capstone_db_euc",

                      collection_name=f'capstone_db_{name}_euc',
                       embedding_function=embedding)
    similarity_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                                              search_kwargs={"k": 5, "score_threshold": 0.0})
    model_name_euc.append(name)
    print(f"Model name: {model_name}")
    #print(embedding.model)
    similarity = similarity_retriever.invoke("What is attention?", k=5)
    retriever_details_euc.append(similarity)
    print(f"Retriever details: {retriever_details}")
    details = dict(zip(model_name, retriever_details))
    combined_list_euc.update(details)

Model name: ['open_ai_large', 'open_ai_small', 'hf']
Retriever details: [[Document(id='e942dafe-4e1a-4565-bf76-d54146766e57', metadata={'page': 2, 'source': '/content/data/attention_paper.pdf'}, page_content='i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3'), Document(id='17c96125-bf85-4013-b3f5-517baf8ee654', metadata={'page': 2, 'source': '/content/data/attention_paper.pdf'}, page_content='i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3'), Document(id='b267b328-a103-4fe6-a4d5-4b37a4bcc165', metadata={'page': 1, 'source': '/content/data/mistral_paper.pdf'}, page_content='we have a theoretical attention span of approximately 131K tokens. In practic

In [ ]:
def create_chroma_db(docs, embedding_list, embedding_name, similarity="cosine"):
  splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  chunked_docs = splitter.split_documents(docs)

  for embedding, name in zip(embedding_list, embedding_names):
    chromba_db = Chroma.from_documents(documents=chunked_docs,
                                      collection_name=f'capstone_db_{name}_{similarity}',
                                      embedding=embedding,
                                      collection_metadata={"hnsw:space": similarity},
                                       persist_directory=f"./capstone_db")
    print(f"Chroma DB created for embedding: {name} with similarity: {similarity}")


In [ ]:
for model, retriever in combined_list_euc.items():
    print(f"Model: {model}")
    for doc in retriever:
        print(f"Document: {doc.page_content}")
        #print(f"Score: {doc.metadata['similarity_score']}")
        print(f"Source: {doc.metadata['source']}")
        print(f"Page: {doc.metadata['page']}")
        print("-" * 80)

Model: open_ai_large
Document: i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3
Source: /content/data/attention_paper.pdf
Page: 2
--------------------------------------------------------------------------------
Document: i. 3.2 Attention An attention function can be described as mapping a query and a set of key-value pairs to an output, where the query, keys, values, and output are all vectors. The output is computed as a weighted sum 3
Source: /content/data/attention_paper.pdf
Page: 2
--------------------------------------------------------------------------------
Document: we have a theoretical attention span of approximately 131K tokens. In practice, for a sequence length of 16K andW = 4096, changes made to FlashAttention [ 11] and xFormers [ 18] yield a 2x speed improvement over a vanilla attention baseline. Rolli

## Cosine similarity is better than Euclidean similarity.  We find the repetition of documents

In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If the answer is not present in the context, just say that you don't know.
            Keep the answer to the point. also show the top 3 context documents of the answer.

            Question:
            {question}

            Context:
            {context}

            Answer:
         """

prompt_template = ChatPromptTemplate.from_template(prompt)

In [ ]:
from langchain_openai import ChatOpenAI

chatgpt = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)


In [ ]:
from langchain_core.runnables import RunnablePassthrough

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_rag_chain = (
    {
        "context": (similarity_retriever
                      |
                    format_docs),
        "question": RunnablePassthrough()
    }
       |
    prompt_template
      |
    chatgpt
)

In [ ]:
from IPython.display import display, Markdown
query = "What is gemini"
result = qa_rag_chain.invoke(query)
display(Markdown(result.content))


Gemini is a family of highly capable multimodal models that includes evaluations at both the model and product levels, focusing on safety mitigations and user experience. It addresses critical policy areas such as hate speech and dangerous content, and adopts a user-centric approach to maximize diversity across various topics and user journeys.

Top 3 context documents:
1. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
2. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced
3. Gemini: A Family of Highly Capable Multimodal Models 7.4.2. Gemini Advanced